In [110]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os, string
import csv
import pymongo
# import time

# Extract (Scrape) Data

In [2]:
# currentDirectory = os.path.dirname(__file__)
# executable_path = {"executable_path": f"{currentDirectory}\chromedriver.exe"}
executable_path = {'executable_path': 'chromedriver.exe'} #notebook version
browser = Browser('chrome', **executable_path, headless=True)

### Scrape Schedules

In [59]:
nba_sched_url = "https://www.espn.com/nba/schedule/_/date/"
browser.visit(nba_sched_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')
sched_tables = soup.find('table', class_="schedule has-team-logos align-left")
schedules_sites = pd.read_html(sched_tables.prettify())
schedules_sites

[            matchup          Unnamed: 1 time  (PT) nat tv  \
 0     Portland  POR        Indiana  IND       LIVE    TNT   
 1   Sacramento  SAC  Oklahoma City  OKC       LIVE    NaN   
 2  Los Angeles  LAL    Golden State  GS    7:30 PM    TNT   
 
                  tickets  Unnamed: 5  
 0     Buy on Vivid Seats         NaN  
 1     Buy on Vivid Seats         NaN  
 2  Tickets as low as $36         NaN  ]

In [ ]:
# for a in string.ascii_lowercase[:26]:
#     print(f"https://www.basketball-reference.com/players/{a}/")

In [ ]:
for letter in string.ascii_lowercase[:26]:
    print(f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm")

In [ ]:
# base_reference_url = "https://www.basketball-reference.com"
# players_url = "https://www.basketball-reference.com/players/a/"
# browser.visit(players_url)
# browser.html
# html = browser.html
# soup = BeautifulSoup(html, 'lxml')

In [61]:
base_reference_url = "https://www.landofbasketball.com"
players_url = "https://www.landofbasketball.com/nba_players_index/letter_h.htm"
browser.visit(players_url)
browser.html 
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [ ]:
all_player_table = soup.find_all('div', class_="indice-items")
for row in all_player_table:
    print(row.find('a')['href'].replace('..', base_reference_url ))

In [20]:
# player_url = "https://www.landofbasketball.com/nba_players/h/rui_hachimura.htm"
# browser.visit(player_url)
# browser.html
# html = browser.html
# soup = BeautifulSoup(html, 'lxml')

In [106]:
tables = pd.read_html(player_url)
print(tables[3][0][0].split(' Profile',1)[0])
try_df = tables[1].set_index(0)
try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
try_df = try_df.to_dict('index')
try_df

Rui Hachimura


{'Team:': {'Value': 'Washington Wizards'},
 'Position:': {'Value': 'Forward'},
 'Jersey Number:': {'Value': '8 (Wizards) - view jerseys...'},
 'Height:': {'Value': '6-8 / 2.03 mts'},
 'Weight:': {'Value': '230 lbs / 104.3 kg'},
 'Born:': {'Value': 'February 08, 1998 in Toyama, Japan'},
 'High-School:': {'Value': 'Meisei in Sendai'},
 'College:': {'Value': 'Gonzaga University'},
 'Drafted:': {'Value': 'by the Washington Wizards with No. 9 overall pick in round 1 of the 2019 NBA Draft'},
 'NBA Seasons:': {'Value': '1 (0 Playoffs)'},
 'NBA Titles:': {'Value': 'none'},
 'All-Star:': {'Value': 'none'}}

## Load dump to mongo

In [111]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.nba
db.nba_players_stg.drop()

In [ ]:
base_reference_url = "https://www.landofbasketball.com"
#Traverse to all players
for letter in string.ascii_lowercase[:26]:
    players_url = f"https://www.landofbasketball.com/nba_players_index/letter_{letter}.htm"
    browser.visit(players_url)
    browser.html
    html = browser.html
    soup = BeautifulSoup(html, 'lxml')
    all_player_table = soup.find_all('div', class_="indice-items")
    for row in all_player_table:
        player_d = {}
        print(row.find('a')['href'], end='')
        player_url = row.find('a')['href'].replace('..', base_reference_url)
        tables = pd.read_html(player_url)
        try_df = tables[1].set_index(0)
        try_df.rename(columns={0: "Key", 1: "Value"}, inplace=True)
        try_df = try_df.to_dict('index')
        player_d['Name'] = tables[3][0][0].split(' Profile',1)[0]
        player_d['Data'] = try_df
        db.nba_players_stg.insert_one(player_d)

../nba_players/a/alaa_abdelnaby.htm../nba_players/a/zaid_abdul_aziz.htm../nba_players/a/kareem_abdul_jabbar.htm../nba_players/a/mahdi_abdul_rahman.htm../nba_players/a/mahmoud_abdul_rauf.htm../nba_players/a/tariq_abdul_wahad.htm../nba_players/a/shareef_abdur_rahim.htm../nba_players/a/tom_abernethy.htm../nba_players/a/forest_able.htm../nba_players/a/john_abramovic.htm../nba_players/a/alex_abrines.htm../nba_players/a/alex_acker.htm../nba_players/a/don_ackerman.htm../nba_players/a/mark_acres.htm../nba_players/a/bud_acton.htm../nba_players/a/quincy_acy.htm../nba_players/a/alvan_adams.htm../nba_players/a/don_adams.htm../nba_players/a/hassan_adams.htm../nba_players/a/jaylen_adams.htm../nba_players/a/jordan_adams.htm../nba_players/a/michael_adams.htm../nba_players/a/steven_adams.htm../nba_players/a/rafael_addison.htm../nba_players/a/bam_adebayo.htm../nba_players/a/deng_adel.htm../nba_players/a/rick_adelman.htm../nba_players/a/jeff_adrien.htm../nba_players/a/arron_afflalo.htm../nba_players/a/ma

In [ ]:
player_list

In [ ]:
# all_player_table = soup.find('div', class_="sortable stats_table now_sortable fixed_cols")
# for row in all_player_table.find_all('th', class_="left"):
#     print(f"{base_reference_url}{row.find('a')['href']}")

In [8]:
# player_url = "https://www.basketball-reference.com/players/a/azubuke01.html"
# browser.visit(player_url)
# browser.html
# html = browser.html
# soup = BeautifulSoup(html, 'lxml')

In [ ]:
bio_div = soup.find_all('div', attrs={"itemtype" : "https://schema.org/Person"})
# print(bio_div[0].h1.text)
# player_list = [a.text.replace('\n', '').strip() for a in bio_div[0].find_all('p')]
# player_list.insert(0, bio_div[0].h1.text)
for prof in bio_div[0].find_all('p'):
        print(prof)

In [10]:
bio_div = soup.find_all('div', attrs={"itemtype" : "https://schema.org/Person"})
# print(bio_div[0].h1.text)
# player_list = [a.text.replace('\n', '').strip() for a in bio_div[0].find_all('p')]
# player_list.insert(0, bio_div[0].h1.text)
for prof in bio_div[0].find_all('p'):
#     prof.strong.decompose()
#     if prof.find('strong'):
#         prof.strong.decompose()
#     print('-------------------------------'+prof.text.replace('\n', '').strip().replace('   ', ''))
    myString = prof.text.replace('\n', '').strip()
    myString = myString.replace('▪','')
#     myString = myString.replace('Twitter:','')
    print('-------------------------------'+" ".join(myString.split()))

# print(player_list)
#         print(a.text.replace('\n', '').strip())

-------------------------------Pronunciation: \keh-LAH-nah ah-zoo-BWEE-kay\
-------------------------------Kelenna David Azubuike Twitter: KAzubuike7
-------------------------------(Big Time Buike)
-------------------------------Position: Small Forward and Shooting Guard Shoots: Right
-------------------------------6-5, 220lb (196cm, 99kg)
-------------------------------Born:December 16, 1983 (Age: 36-072d) in London, United Kingdomgb
-------------------------------College: Kentucky
-------------------------------High School: Victory Christian in Tulsa, Oklahoma
-------------------------------Recruiting Rank: 2002 (31)
-------------------------------NBA Debut: January 2, 2007


In [ ]:
output_file = os.path.join("nba_players.csv")
#  Open the output file
with open(output_file, "w", newline="") as datafile:
    writer = csv.writer(datafile)
    base_reference_url = "https://www.basketball-reference.com"
    all_players = []
    for letter in string.ascii_lowercase[:26]:
        players_url = f"https://www.basketball-reference.com/players/{letter}/"
        browser.visit(players_url)
        browser.html
        html = browser.html
        soup = BeautifulSoup(html, 'lxml')
        all_player_table = soup.find('table', class_="sortable stats_table now_sortable fixed_cols")
        for row in all_player_table.find_all('th', class_="left"):
            print(f"{base_reference_url}{row.find('a')['href']}")
            player_url = f"{base_reference_url}{row.find('a')['href']}"
            browser.visit(player_url)
            browser.html
            html = browser.html
            soup = BeautifulSoup(html, 'lxml')
            bio_div = soup.find_all('div', attrs={"itemtype" : "https://schema.org/Person"})
            player_list = []
    #         player_list = [a.text.replace('\n', '').strip() for a in bio_div[0].find_all('p')]
    #         player_list.insert(0, bio_div[0].h1.text)
            for prof in bio_div[0].find_all('p'):
                myString = prof.text.replace('\n', '').strip()
                myString = myString.replace('▪','')
                player_list.append(" ".join(myString.split()))
            player_list.insert(0, bio_div[0].h1.text)
#             print(player_list)
#             all_players.append(player_list)
            writer.writerow(player_list)

In [12]:
all_players

[['Alaa Abdelnaby',
  'Alaa Abdelnaby Twitter: alaatweets',
  '(Alphabet, The Pharaoh, Black Hole)',
  'Position: Power Forward Shoots: Right',
  '6-10, 240lb (208cm, 108kg)',
  'Born:June 24, 1968 (Age: 51-247d) in Cairo, Egypteg',
  'College: Duke',
  'High School: Bloomfield in Bloomfield, New Jersey',
  'Draft: Portland Trail Blazers, 1st round (25th pick, 25th overall), 1990 NBA Draft',
  'NBA Debut: November 2, 1990']]

In [ ]:
df

In [ ]:
base_image_url = 'https://www.jpl.nasa.gov'
target_image_url = f'{base_image_url}/spaceimages/?search=&category=Mars'
browser.visit(target_image_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')

In [ ]:
image_art = soup.find('ul', class_='articles')
images = image_art.find_all('li', class_='slide')

for image in images:
    featured_image_url = f"{base_image_url}{image.a['data-fancybox-href']}"
    print(featured_image_url)
    print(20*'---')

In [ ]:
twitter_url = 'https://twitter.com/marswxreport'
response = requests.get(twitter_url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
tweets_div = soup.find_all('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")

for tweet in tweets_div:
    if tweet.text.startswith("InSight sol", 0):
        print(tweet.text)
    else:
        continue
    print(20*'---')

In [ ]:
facts_site = pd.read_html("https://space-facts.com/mars/")

In [ ]:
mars_df = pd.DataFrame(facts_site[0])
mars_df.set_index(0)
new_dict = {}
for row in mars_df.iterrows():
    new_dict[row[1][0]] = row[1][1]
new_dict

In [ ]:
usgs_image_url = 'https://astropedia.astrogeology.usgs.gov'
base_image_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# target_image_url = f'{base_image_url}/spaceimages/?search=&category=Mars'
browser.visit(base_image_url)
browser.html
html = browser.html
soup = BeautifulSoup(html, 'lxml')


In [ ]:
pic_links = soup.find_all('a', class_="itemLink")
# http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif

In [ ]:
hemisphere_image_urls = []
for pic in pic_links:
    img = {}
    if pic.h3:
        img['title'] = pic.h3.text
        img['img_url'] = usgs_image_url+pic['href'].replace('/search/map/','/download/') +'.tif'
        hemisphere_image_urls.append(img)

In [ ]:
hemisphere_image_urls

In [11]:
all_players[5]

['Kelenna Azubuike',
 'Pronunciation: \\keh-LAH-nah ah-zoo-BWEE-kay\\',
 'Kelenna David Azubuike      ▪  Twitter:  KAzubuike7',
 '(Big Time Buike)',
 'Position:    Small Forward and Shooting Guard    ▪      Shoots:    Right',
 '6-5,\xa0220lb\xa0(196cm,\xa099kg)',
 'Born:December 16,            1983 (Age:\xa036-072d)    in\xa0London,\xa0United Kingdomgb',
 'College:        Kentucky',
 'High School:    Victory Christian  in  Tulsa,        Oklahoma',
 'Recruiting Rank:  2002 (31)',
 'NBA Debut: January 2, 2007']